In [ ]:
!pip install --upgrade --quiet  google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client langchain langchain-community langchain-openai langchainhub

### Create the Gmail Toolkit from LangChain

In [ ]:
from langchain_community.agent_toolkits import GmailToolkit
from langchain_community.tools.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)


credentials = get_gmail_credentials(
    scopes=["https://www.googleapis.com/auth/gmail.readonly"],
    client_secrets_file="/content/web_credentials.json",
    token_file = "/content/token.json"
)

print("credentials Generated....")
api_resource = build_resource_service(credentials=credentials)
toolkit = GmailToolkit(api_resource=api_resource)

credentials Generated....


In [ ]:
tools = toolkit.get_tools()
tools

[GmailCreateDraft(api_resource=<googleapiclient.discovery.Resource object at 0x7878d7c935e0>),
 GmailSendMessage(api_resource=<googleapiclient.discovery.Resource object at 0x7878d7c935e0>),
 GmailSearch(api_resource=<googleapiclient.discovery.Resource object at 0x7878d7c935e0>),
 GmailGetMessage(api_resource=<googleapiclient.discovery.Resource object at 0x7878d7c935e0>),
 GmailGetThread(api_resource=<googleapiclient.discovery.Resource object at 0x7878d7c935e0>)]

### Passing your Openai API Key

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


### Working with Gmails with OpenAI Tools Agent

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

In [ ]:
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [ ]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'instructions': 'You are an assistant.'}, metadata={'lc_hub_owner': 'langchain-ai', 'lc_hub_repo': 'openai-functions-template', 'lc_hub_commit_hash': 'b4198088b0f11cbe911a9e1cb6546893c00283b6cf0f80ae003b3955efdfe281'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_va

In [ ]:
llm = ChatOpenAI(temperature=0)

In [ ]:
agent = create_openai_tools_agent(llm, toolkit.get_tools(), prompt)

In [ ]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=toolkit.get_tools(),
    # This is set to False to prevent information about my email showing up on the screen
    # Normally, it is helpful to have it set to True however.
    verbose=True,
)

In [ ]:
agent_executor.invoke(
    {
        "input": "Create a gmail draft for me to reschedule my meeting from sunday to morning to monday morning"
        " as I will be not available because of a football match. The recipient is rounak.1210@gmail.com"
        " keep the tone of the email polite"
    }
)



> Entering new AgentExecutor chain...

Invoking: `create_gmail_draft` with `{'message': 'Dear Rounak, \n\nI hope this message finds you well. I wanted to inform you that I have a prior commitment on Sunday morning and will not be available for our meeting. Would it be possible to reschedule our meeting to Monday morning instead? I apologize for any inconvenience this may cause. Thank you for your understanding. \n\nBest regards, [Your Name]', 'to': ['rounak.1210@gmail.com'], 'subject': 'Meeting Rescheduling Request'}`


Draft created. Draft Id: r-922369640620305716I have created a draft email for you to reschedule your meeting from Sunday morning to Monday morning. The draft has been saved, and you can review and send it from your Gmail account.

> Finished chain.


{'input': 'Create a gmail draft for me to reschedule my meeting from sunday to morning to monday morning as I will be not available because of a football match. The recipient is rounak.1210@gmail.com keep the tone of the email polite',
 'output': 'I have created a draft email for you to reschedule your meeting from Sunday morning to Monday morning. The draft has been saved, and you can review and send it from your Gmail account.'}

In [ ]:
agent_executor.invoke(
    {"input": "Could you search in primary inbox for the latest email? what is the title?"}
)



> Entering new AgentExecutor chain...

Invoking: `search_gmail` with `{'query': 'label:inbox category:primary', 'max_results': 1}`


[{'id': '18e609ee4cb6fb88', 'threadId': '18e609ee4cb6fb88', 'snippet': '[11/25/2023, 12:06] Rik: Developed a Python-based NLP tool using Google API and BERT for sentiment analysis of YouTube video comments. I scraped comments from YouTube video URLs, conducted contextual', 'body': '[11/25/2023, 12:06] Rik: Developed a Python-based NLP tool using Google API\r\nand BERT for sentiment analysis of YouTube video comments. I scraped\r\ncomments from YouTube video URLs, conducted contextual sentiment analysis,\r\nand predicted sentiment as star ratings (1-5).\r\n[11/25/2023, 12:08] Rik: Moreover, my News QnA Analyzer Tool project\r\ninvolved implementing advanced NLP techniques. I developed and implemented\r\nan automated content analysis tool that processed URLs of news articles,\r\ndelivering comprehensive answers to user-generated\r\nquestions; reduced rese

{'input': 'Could you search in primary inbox for the latest email? what is the title?',
 'output': 'The latest email in your primary inbox has the title "Sample msg".'}

### Pass Custom Prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
You are a helpful assistant.
"""),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,
        return_intermediate_steps=True
    )

response = agent_executor.invoke(
                    {
                        "input": user_query,
                        "chat_history": chat_history,
                    }
    )
